# Approach
- Basic Data Analysis
- Exploratory Data Analysis
    - Univariate, Bivariate, Multivariate Analysis
    - Target vs Features
- Data Preprocessing
- Statistical Tests
    - Chisquare, Anova Test
- Modeling
- Assumptions of Linear Regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, precision_recall_curve, auc

# from xgboost import XGBClassifier
bold_start = '\033[1m'
bold_end = '\033[0m'

In [2]:
raw_data = pd.read_csv(r"F:\Muthu_2023\Personal\NextStep\DSCourse\Scaler\Business-Case-Study\Scaler\Dataset\scaler_clustering.csv")
raw_data.head()

,Unnamed: 0,company_hash,email_hash,orgyear,ctc,job_position,ctc_updated_year
0,0,atrgxnnt xzaxv,6de0a4417d18ab14334c3f43397fc13b30c35149d70c05...,2016.0,1100000,Other,2020.0
1,1,qtrxvzwt xzegwgbb rxbxnta,b0aaf1ac138b53cb6e039ba2c3d6604a250d02d5145c10...,2018.0,449999,FullStack Engineer,2019.0
2,2,ojzwnvwnxw vx,4860c670bcd48fb96c02a4b0ae3608ae6fdd98176112e9...,2015.0,2000000,Backend Engineer,2020.0
3,3,ngpgutaxv,effdede7a2e7c2af664c8a31d9346385016128d66bbc58...,2017.0,700000,Backend Engineer,2019.0
4,4,qxen sqghu,6ff54e709262f55cb999a1c1db8436cb2055d8f79ab520...,2017.0,1400000,FullStack Engineer,2019.0


# Data Analysis

In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205843 entries, 0 to 205842
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        205843 non-null  int64  
 1   company_hash      205799 non-null  object 
 2   email_hash        205843 non-null  object 
 3   orgyear           205757 non-null  float64
 4   ctc               205843 non-null  int64  
 5   job_position      153281 non-null  object 
 6   ctc_updated_year  205843 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 11.0+ MB


In [5]:
raw_data.describe()

,Unnamed: 0,orgyear,ctc,ctc_updated_year
count,205843.000000,205757.000000,2.058430e+05,205843.000000
mean,103273.941786,2014.882750,2.271685e+06,2019.628231
std,59741.306484,63.571115,1.180091e+07,1.325104
min,0.000000,0.000000,2.000000e+00,2015.000000
25%,51518.500000,2013.000000,5.300000e+05,2019.000000
50%,103151.000000,2016.000000,9.500000e+05,2020.000000
75%,154992.500000,2018.000000,1.700000e+06,2021.000000
max,206922.000000,20165.000000,1.000150e+09,2021.000000


In [6]:
raw_data.describe(include='object')

,company_hash,email_hash,job_position
count,205799,205843,153281
unique,37299,153443,1017
top,nvnv wgzohrnvzwj otqcxwto,bbace3cc586400bbc65765bc6a16b77d8913836cfc98b7...,Backend Engineer
freq,8337,10,43554


## Unique Values

In [7]:
raw_data.nunique()

Unnamed: 0          205843
company_hash         37299
email_hash          153443
orgyear                 77
ctc                   3360
job_position          1017
ctc_updated_year         7
dtype: int64

## Null Values

In [12]:
print(bold_start + "Percentage of Null values by columns" + bold_end)
(raw_data.isnull().sum()/len(raw_data))*100

Percentage of Null values by columns


Unnamed: 0           0.000000
company_hash         0.021376
email_hash           0.000000
orgyear              0.041779
ctc                  0.000000
job_position        25.534995
ctc_updated_year     0.000000
dtype: float64

#### Inference:
- Dimensionally small dataset
- Personal details are given as hash values
- Some errors in org year column, as the min value is 0 and no. of unique values is 77

## Transform Hash

In [13]:
df = raw_data.copy()

In [183]:
company_hash_dict = dict(zip(df['company_hash'].unique(), np.arange(df['company_hash'].nunique())))
df['company_hash'] = df['company_hash'].map(company_hash_dict)

In [22]:
email_hash_dict = dict(zip(df['email_hash'].unique(), np.arange(df['email_hash'].nunique())))
df['email_hash'] = df['email_hash'].map(email_hash_dict)

## Remove Unnamed column

In [24]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.head()

# Exploratory Data Analysis
- Univariate Analysis
    - Categorical: *Frequency and Proportion Plots*
    - Numerical: *Distribution and Box plots*
    - *Skewness and Kurtosis*
- Bivariate Analysis
- Multivariate Analysis

In [25]:
cat_features = list(df.columns).remove('ctc')
num_features = ['ctc']

In [ ]:
def univariate_categoryplot(df, feature):
    plt.figure(figsize=(12,4))   
    if df[feature].nunique() < 10:
        # Categorical Feature 
        plt.suptitle('Univariate Analysis of ' + feature + ": Frequency and Proportion")
        plt.subplot(1,2,1)
        sns.countplot(data=df, x=feature) 
        plt.subplot(1,2,2)
        labels = df[feature].value_counts().index
        plt.pie(x=list(df[feature].value_counts()), labels=list(labels), autopct="%0.2f")        
        plt.show()
    else:
        # Numerical Feature
        temp = df[feature].value_counts()[:10]
        plt.suptitle('Univariate Analysis of ' + feature + ": Frequency and Proportion")
        plt.subplot(1,2,1)
        sns.countplot(data=df, x=feature) 
        plt.subplot(1,2,2)
        labels = df[feature].value_counts().index
        plt.pie(x=list(df[feature].value_counts()), labels=list(labels), autopct="%0.2f")        
        plt.show()

# Data Cleaning and Transformation

## Orgyear

In [42]:
np.set_printoptions(suppress=True)
print(df['orgyear'].unique())

[ 2016.  2018.  2015.  2017.  2019.  2020.  2012.  2013.  2003.  2006.
  2014.  2011.  2021.  2008.  2004.  2022.  2009.  2005.  2010.  2007.
  2000.  2002.  2023.  2001.  1981.  2031.    nan  2024.  1996.  1999.
  2106.  1997.  1994.  1995.  1992.  1973.  1991.  1998.  1990.  1993.
  1988.  2025.  2029.     0.   208.  1985.   209.   206.  1982.  2026.
  1970.  2101.  1972.  2107.  1986.  1989.    91.  1987.     3.  2027.
     2.  1976.     4.     5.  1971.  1977.  1984.    83.     1.  1979.
  2028.  2204.    38.  1900.   201.     6. 20165.   200.]


In [40]:
df['orgyear'] = pd.to_datetime(df['orgyear'].apply(lambda x: x if x < 2025 and x>1900 else np.nan), format='%Y').dt.year

173

#### Inference:
- Many typo errors in the year column such as 0, 1, 2, 209, 91 etc.,
- Transformed such typo errors into nan values
- Later all nan values will be imputed

## CTC Updated Year

In [43]:
np.set_printoptions(suppress=True)
print(df['ctc_updated_year'].unique())

[2020. 2019. 2021. 2017. 2016. 2015. 2018.]


In [40]:
df['orgyear'] = pd.to_datetime(df['orgyear'].apply(lambda x: x if x < 2025 and x>1900 else np.nan), format='%Y').dt.year

173

#### Inference:
- Many typo errors in the year column such as 0, 1, 2, 209, 91 etc.,
- Transformed such typo errors into nan values
- Later all nan values will be imputed

## Job Position

In [133]:
sorted(df['job_position'].dropna().unique())[-500:]

['Project engineer',
 'Project management profession',
 'Pune',
 'Python Devloper',
 'Q',
 'QA Engineer',
 'QA intern',
 'QA/QC Engineer',
 'QAE-Intern',
 'Quality Analyst',
 'Quality Analyst ',
 'Quality Associate',
 'Quality Assurance Manager',
 'Quality Engineer',
 'Quality Enginner',
 'Quantitative Analyst',
 'R & D Engineer-2',
 'R & D engineer',
 'R&D Engineer',
 'RD Engineer II',
 'RD Enginner II',
 'RETAIL ASSCOCIATE',
 'Radio frequency Engineer',
 'React Native developer',
 'Reasearch Engineer',
 'Recovery officer',
 'Relationship officer',
 'Release Engineer',
 'Reseach Analyst',
 'Research Assistant',
 'Research Assistant / Software Engineer',
 'Research Engineer',
 'Research Engineer 2',
 'Research Engineers',
 'Research Intern - Machine Learning',
 'Research analyst',
 'Researcher',
 'Reseller',
 'Risk Analyst',
 'Risk Investigator',
 'Risk model developer',
 'S/w Engineer 2',
 'SAP ABAP Developer',
 'SAP C4C Functional Consultant',
 'SAP CPQ Consultant',
 'SAP Technical c

#### Inference:
- Invalid occupations are observed
- Case sensitive differences
- Replace special characters with space
- Trim the text and remove double spaces
- Remove only numbers
- create categories for occupation
- Software Categories: Software, application, system, systems, IT, tech, technology, frontend, front end, backend, full stack, fullstack, cloud, computer, database, devops, firmware, app, programmer, java, developer, UI, sde, sw, se, web
- Data categories: data, analyst, analytics, business, etl, ML, machine learning, ai
- Not employed: 'no', 'none', 'not', 'null'
- other to others
- Search for non dictionary words

In [ ]:
sw_cat = ['software', 'application', 'system', 'systems', 'IT', 'tech', 'technology', 'frontend', 'end', 'backend', 'stack', 'fullstack', 'cloud', 'computer', 'database', 'devops', 'firmware', 'app', 'programmer', 'java', 'developer', 'ui', 'sde', 'sw', 'se', 'web']
data_cat = ['data', 'analyst', 'analytics', 'business', 'etl', 'ml', 'machine', 'learning', 'ai']

In [164]:
def text_clean(text):    
    if type(text) == str:
        text = text.lower().strip()
        text = re.sub('[^A-Za-z0-9 ]+', ' ', text)
        text = text.replace("  ", " ").strip()
        text = re.sub(r'^\d+$', 'others', text)
        if text == '':
            text = 'others'
        words = text.split()
        for word in words:
            if word in ['no', 'none', 'not', 'null', 'some']:
                return "Not Employed"
    return text

In [171]:
def assign_software(text):
    if type(text) == str:
        for word in text.split():
            if word in sw_cat:
                return 1
    return 0

In [179]:
def assign_data(text):
    if type(text) == str:
        for word in text.split():
            if word in data_cat:
                return 1
    return 0

In [166]:
df['job_cleaned'] = df['job_position'].apply(text_clean)
df['software_job'] = df['job_cleaned'].apply(assign_software)
df['data_job'] = df['job_cleaned'].apply(assign_data)

In [185]:
# Drop raw job position column
df.drop('job_position', axis=1, inplace=True)

In [186]:
df.head()

,company_hash,email_hash,orgyear,ctc,ctc_updated_year,job_cleaned,software_job,data_job
0,0.0,0,2016.0,1100000,2020.0,other,0,0
1,1.0,1,2018.0,449999,2019.0,fullstack engineer,1,0
2,2.0,2,2015.0,2000000,2020.0,backend engineer,1,0
3,3.0,3,2017.0,700000,2019.0,backend engineer,1,0
4,4.0,4,2017.0,1400000,2019.0,fullstack engineer,1,0


In [187]:
df.isnull().sum()

company_hash            1
email_hash              0
orgyear                86
ctc                     0
ctc_updated_year        0
job_cleaned         52562
software_job            0
data_job                0
dtype: int64

In [190]:
raw_data.groupby('email_hash')['orgyear'].apply(list)

email_hash
00003288036a44374976948c327f246fdbdf077854690475243a9c91d3b1cf9f            [2012.0]
0000aaa0e6b61f7636af1954b43d294484cd151c9b3cf6a761391159aa77f3fa            [2013.0]
0000d58fbc18012bf6fa2605a7b0357d126ee69bc41032ace6b6223c48468887    [2021.0, 2021.0]
000120d0c8aa304fcf12ab4b85e21feb80a342cfea03d461a50a3e16626a4177            [2004.0]
00014d71a389170e668ba96ae8e1f9d991591acc8990253cc62a371eea92155b            [2009.0]
                                                                          ...       
fffc254e627e4bd1bc0ed7f01f9aebbba7c3cc56ac914ebadddd76333a7855fc            [2004.0]
fffcf97db1e9c13898f4eb4cd1c2fe862358480e104535a548f1b9daf660771c            [2015.0]
fffe7552892f8ca5fb8647d49ca805b72ea0e9538b6b015e59d649b349641174            [2014.0]
ffff49f963e4493d8bbc7cc15365423d84a767259f7200a11548e704e156d898            [2020.0]
ffffa3eb3575f43b86d986911463dce7bcadcea227e5a4b0f1756744efdb7477            [2018.0]
Name: orgyear, Length: 153443, dtype: object

In [211]:
def test(val):
    list_values = list(val)
#     return list_values
    for value in list_values:
        if math.isnan(value):
            return 1        
    return 0

In [216]:
test_data = raw_data.groupby('email_hash')['orgyear'].apply(list)

In [224]:
test_data[temp==1]

email_hash
01a284abb1ed97d72506b7a3f641b9b545e1878dfe9e5021859ab9f95b654114            [nan]
040a1f04539f587f7c656912fe42d03617f2f65b33c2aa8026dfe38ab15664b2            [nan]
060f40a7586f40c79211d1d6ba97b928d16991dfb9f4cb8cdbb2a4e19c5ced7e            [nan]
0e25bf875aaa6550db1aa9f9452179ed1cd6737c079fbc9ae3056adcee011189    [nan, 2017.0]
0f4c60c23f3161fa21002935631eeb51f3d94aaa75fee7c33937c4641799889e            [nan]
                                                                        ...      
f0e7f60ee9d43d575b80afd9b24057c303d2a2a2e0877db4c6f0304dbe88f5fe            [nan]
f7b814c6c4b821b6293ae3f9789c8839f3b0f8666b1d02917cc132315c74fd2c            [nan]
f9c1d0e623046d445a705bc39007f05619bcd37c75d3e8a9f840beda979d776d            [nan]
fadfe9dd062538ab3c2069a0d60aae2752ed0f513c79af886b5dbbd88c92c631            [nan]
fcea167cdfdb1e2f23a2733ab591e6a42bda2e5560826e18c5174d62b4f2cda5            [nan]
Name: orgyear, Length: 84, dtype: object

In [215]:
temp[temp==1]

email_hash
01a284abb1ed97d72506b7a3f641b9b545e1878dfe9e5021859ab9f95b654114    1
040a1f04539f587f7c656912fe42d03617f2f65b33c2aa8026dfe38ab15664b2    1
060f40a7586f40c79211d1d6ba97b928d16991dfb9f4cb8cdbb2a4e19c5ced7e    1
0e25bf875aaa6550db1aa9f9452179ed1cd6737c079fbc9ae3056adcee011189    1
0f4c60c23f3161fa21002935631eeb51f3d94aaa75fee7c33937c4641799889e    1
                                                                   ..
f0e7f60ee9d43d575b80afd9b24057c303d2a2a2e0877db4c6f0304dbe88f5fe    1
f7b814c6c4b821b6293ae3f9789c8839f3b0f8666b1d02917cc132315c74fd2c    1
f9c1d0e623046d445a705bc39007f05619bcd37c75d3e8a9f840beda979d776d    1
fadfe9dd062538ab3c2069a0d60aae2752ed0f513c79af886b5dbbd88c92c631    1
fcea167cdfdb1e2f23a2733ab591e6a42bda2e5560826e18c5174d62b4f2cda5    1
Name: orgyear, Length: 84, dtype: int64

## Knn Imputation for Missing Values